In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession. \
    builder. \
    config("spark.sql.warehouse.dir", "/user/forgcpmak/warehouse").\
    enableHiveSupport(). \
    appName('Spark SQL - Managing Tables - DML and Partitioning'). \
    master('yarn'). \
    getOrCreate()

In [5]:
spark

In [ ]:
spark.sql('SHOW databases').show()

In [ ]:
spark.sql("use retail_db")

In [ ]:
spark.sql("show functions").show(300, False)

In [ ]:
spark.sql("DESCRIBE FUNCTION substr").show(truncate = False)

In [ ]:
spark.sql("select current_date").show(truncate = False)

In [13]:
ordersdf = spark.read. \
    schema("order_id INT, order_date STRING, order_customer_id INT, order_status STRING"). \
    csv("/user/forgcpmak/retail_db/orders")
ordersdf.createOrReplaceTempView("orders_temp")

In [53]:
spark.sql("""
SELECT to_unix_timestamp('2019-04-30 18:18:51') AS unixtime
    """).show(truncate = False)

+----------+
|unixtime  |
+----------+
|1556648331|
+----------+



In [69]:
spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS orders_single_column (
    s STRING
) LOCATION '/user/hive/warehouse/retail_db.db/orders'
    """).show(truncate = False)

++
||
++
++



In [71]:
spark.sql("""
SELECT split(s, ',')[0] AS order_id,
    split(s, ',')[1] AS order_date,
    split(s, ',')[2] AS order_customer_id,
    split(s, ',')[3] AS order_status
FROM orders_single_column LIMIT 10
""").show(truncate = False)

+--------+----------+-----------------+------------+
|order_id|order_date|order_customer_id|order_status|
+--------+----------+-----------------+------------+
+--------+----------+-----------------+------------+



In [ ]:
spark.sql("""
select count(1) from (SELECT explode(split('2013-07-25', '-')) AS result) q
    """).show(truncate = False)

In [31]:
spark.sql("""
SELECT concat_ws('-', year, lpad(month, 2, 0),
              lpad(myDate, 2, 0)) AS order_date
FROM
    (SELECT 2013 AS year, 7 AS month, 25 AS myDate) q
    """).show(truncate = False)

+----------+
|order_date|
+----------+
|2013-07-25|
+----------+



In [ ]:
spark.sql('show tables').show()

In [ ]:
spark.sql('DROP TABLE orders').show()

In [ ]:
spark.sql('SELECT current_database()').show()

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS retail_db.orders (
  order_id INT,
  order_date STRING,
  order_customer_id INT,
  order_status STRING
) 
ROW FORMAT 
     DELIMITED FIELDS TERMINATED BY ','
"""
         )

In [ ]:
spark.sql("""
LOAD DATA LOCAL INPATH '/home/forgcpmak/retail_db/orders'
    OVERWRITE INTO TABLE orders
    """)

In [ ]:
spark.sql('SELECT count(1) FROM retail_db.orders').show()

In [ ]:
spark.sql('DROP TABLE IF EXISTS order_items').show()

In [ ]:
spark.sql("""
CREATE TABLE order_items (
  order_item_id INT,
  order_item_order_id INT,
  order_item_product_id INT,
  order_item_quantity INT,
  order_item_subtotal FLOAT,
  order_item_product_price FLOAT
) STORED AS parquet
          
          """).show()

In [ ]:
spark.sql("DESCRIBE FORMATTED order_items").show(truncate = False)

In [ ]:
spark.sql("""
CREATE TABLE If NOT EXISTS retail_db.order_items_stage (
  order_item_id INT,
  order_item_order_id INT,
  order_item_product_id INT,
  order_item_quantity INT,
  order_item_subtotal FLOAT,
  order_item_product_price FLOAT
) ROW FORMAT 
   DELIMITED FIELDS TERMINATED BY ','
   """)


In [ ]:
spark.sql("select * from retail_db.order_items_stage").show()

In [ ]:
spark.sql("LOAD DATA LOCAL INPATH '/home/forgcpmak/retail_db/order_items' INTO TABLE retail_db.order_items_stage")

In [ ]:
spark.sql("SELECT count(1) FROM retail_db.order_items_stage").show()

In [ ]:
spark.sql("""
INSERT INTO TABLE order_items
SELECT * FROM order_items_stage
"""
         ).show()


In [ ]:
spark.sql("SELECT count(1) FROM order_items").show()

In [ ]:
spark.sql("""
INSERT INTO TABLE order_items
SELECT * FROM order_items_stage
"""
         ).show()

In [ ]:
spark.sql("SELECT count(1) FROM retail_db.order_items").show()

In [ ]:
spark.sql("""
INSERT OVERWRITE TABLE order_items
SELECT * FROM order_items_stage
"""
         ).show()

In [ ]:
spark.sql("SELECT count(1) FROM retail_db.order_items").show()

In [ ]:
--- this completes the loading data from staging to parquet table ----

In [ ]:
---- start for partition table creation. ----


In [ ]:
spark.sql("""DROP TABLE IF EXISTS orders_part""")

In [ ]:
spark.sql("""
CREATE TABLE orders_part (
  order_id INT,
  order_date STRING,
  order_customer_id INT,
  order_status STRING
) PARTITIONED BY (order_month INT)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
"""
         )

In [ ]:
spark.sql("SELECT * FROM orders_part").show(truncate = False)

In [ ]:
spark.sql("DESCRIBE FORMATTED orders_part").show(200, False)

In [ ]:
spark.sql("""
ALTER TABLE orders_part ADD
    PARTITION (order_month=201308)
    PARTITION (order_month=201309)
    PARTITION (order_month=201310)
    """)

In [ ]:
spark.sql("""
CREATE TABLE orders_part_string (
  order_id INT,
  order_date STRING,
  order_customer_id INT,
  order_status STRING
) PARTITIONED BY (order_month STRING)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
""")

In [ ]:
spark.sql("""
ALTER TABLE orders_part_string ADD PARTITION (order_month='2013-07')
""")

In [ ]:
spark.sql("""
ALTER TABLE orders_part_string ADD
    PARTITION (order_month='201308')
    PARTITION (order_month='201309')
    PARTITION (order_month='201310')
    """)

In [ ]:
spark.sql("""SET hive.exec.dynamic.partition=true""")

In [ ]:
spark.sql("""SET hive.exec.dynamic.partition.mode=nonstrict""")

In [ ]:
spark.sql("""
INSERT INTO TABLE orders_part_string PARTITION (order_month)
SELECT o.*, date_format(order_date, 'yyyyMM') order_month
FROM orders o
WHERE order_date >= '2013-12-01 00:00:00.0'
"""
         )

In [ ]:
spark.sql("""
select count(1) from orders_part_string
""").show()

In [ ]:
--- start from here for spark sql functions -----

In [ ]:
spark.sql("""
CREATE EXTERNAL TABLE orders (
  order_id INT COMMENT 'Unique order id',
  order_date STRING COMMENT 'Date on which order is placed',
  order_customer_id INT COMMENT 'Customer id who placed the order',
  order_status STRING COMMENT 'Current status of the order'
) COMMENT 'Table to save order level details'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LOCATION '/user/forgcpmak/external/retail_db/orders'
"""
         ).show()

In [ ]:
spark.sql("""
CREATE EXTERNAL TABLE orders_in_parquet (
  order_id INT COMMENT 'Unique order id',
  order_date STRING COMMENT 'Date on which order is placed',
  order_customer_id INT COMMENT 'Customer id who placed the order',
  order_status STRING COMMENT 'Current status of the order'
) COMMENT 'Table to save order level details'
STORED AS parquet
LOCATION '/user/forgcpmak/external/retail_db/orders_in_parquet'
"""
         ).show()

In [ ]:
e=spark.read.text("/user/forgcpmak/retail_db/orders")

df = spark.read.csv('/user/forgcpmak/retail_db/orders', header = False)
df.repartition(1).write.mode('overwrite').parquet('/user/forgcpmak/retail_db/orders_parquet')

In [ ]:
e.show(truncate = False)

In [ ]:
df.show()

In [ ]:
df.write.format("parquet").insertInto("orders_in_parquet") 

In [ ]:
from pyspark.sql.functions import split,col

spark.sql("SET hive.mapred.supports.subdirectories=TRUE")
spark.sql("SET mapred.input.dir.recursive=TRUE")

In [ ]:
spark.sql('SET hive.support.quoted.identifiers=none')

In [ ]:
spark.sql("LOAD DATA  INPATH '/user/forgcpmak/retail_db/orders_parquet/part-00000-37a022b9-4666-4a31-b27b-9628ed5cee90-c000.snappy.parquet' INTO TABLE orders_in_parquet")

In [ ]:
spark.sql("DESCRIBE extended orders_in_parquet").show(truncate = False)

In [ ]:
spark.sql("select * from orders_in_parquet").show(truncate = False)

In [ ]:
spark.sql("""
SELECT order_customer_id, 
    date_format(order_date, 'yyyy-MM'), 
    order_status 
FROM orders""").show()

In [ ]:
spark.sql("""SELECT * FROM orders WHERE order_status = 'COMPLETE' LIMIT 10""").show(truncate = False)

In [ ]:
spark.sql("SELECT * FROM orders WHERE order_status IN ('COMPLETE', 'CLOSED')").show(truncate = False)

In [ ]:
spark.sql("""
SELECT count(1) FROM orders
WHERE order_status = 'COMPLETE' OR order_status = 'CLOSED'
""").show(truncate = False)

In [ ]:
spark.sql("""
SELECT * FROM orders 
WHERE order_status IN ('COMPLETE', 'CLOSED')
    AND date_format(order_date, 'yyyy-MM') = '2014-01'
""").show(truncate = False)

In [ ]:
spark.sql("""
SELECT * FROM orders 
WHERE order_status IN ('COMPLETE', 'CLOSED')
    AND order_date LIKE '2014-01%'
""").show(truncate = False)

In [ ]:
spark.sql("DROP DATABASE IF EXISTS forgcpmak_sms CASCADE")

In [ ]:
spark.sql("CREATE DATABASE IF NOT EXISTS  forgcpmak_sms")

In [ ]:
spark.sql("""
CREATE TABLE students (
    student_id INT,
    student_first_name STRING,
    student_last_name STRING,
    student_phone_number STRING,
    student_address STRING
) STORED AS avro
""")

In [ ]:
spark.sql("""
INSERT INTO students VALUES (1, 'Scott', 'Tiger', NULL, NULL)
""").show()

In [ ]:
spark.sql("""
INSERT INTO students VALUES
   (2, 'Donald', 'Duck', '1234567890', NULL),
    (3, 'Mickey', 'Mouse', '2345678901', 'A Street, One City, Some State, 12345'),
    (4, 'Bubble', 'Guppy', '6789012345', 'Bubbly Street, Guppy, La la land, 45678')
   """).show()


In [ ]:
 spark.sql("SELECT * FROM students").show()

In [ ]:
spark.sql("""
SELECT * FROM students 
WHERE student_phone_number = NULL
""").show()

In [ ]:
spark.sql("""
SELECT * FROM students
WHERE student_phone_number IS NULL
""").show()

In [ ]:
spark.sql("""
SELECT o.order_id,
    o.order_date,
    o.order_status,
    oi.order_item_subtotal
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
LIMIT 10
""").show(truncate = False)

In [ ]:
spark.sql("""
SELECT o.order_id,
    o.order_date,
    o.order_status,
    oi.order_item_subtotal
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status != 'PENDING_PAYMENT'
LIMIT 10
""").show(truncate = False)

In [ ]:
spark.sql("""
SELECT o.order_id,
    o.order_date,
    o.order_status,
    oi.order_item_subtotal
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status != 'PENDING_PAYMENT'
LIMIT 10
""").show(truncate = False)

In [ ]:
spark.sql("SELECT count(DISTINCT order_date) FROM orders").show()

In [ ]:
spark.sql("""
SELECT round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items 
WHERE order_item_order_id = 2
""").show(truncate = False)

In [ ]:
spark.sql("""
SELECT order_date,
    count(1)
FROM orders
GROUP BY order_date
having count(1) > 50
order by 2
""").show(truncate = False)

In [ ]:
spark.sql("""
SELECT o.order_date,
       oi.order_item_product_id,
       round(sum(oi.order_item_subtotal), 2) AS revenue
FROM   orders o 

JOIN order_items oi
ON o.order_id = oi.order_item_order_id

WHERE o.order_status IN ('COMPLETE', 'CLOSED')

GROUP BY 
        o.order_date,
       oi.order_item_product_id
""").show(truncate = False)

In [ ]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
ORDER BY o.order_date,
    revenue DESC
""").show()

In [ ]:
spark.sql("describe extended orders").show(truncate = False);

In [ ]:
spark.sql("""
SELECT * FROM orders
ORDER BY order_customer_id,
    order_date DESC
""").show(truncate = False)

In [ ]:
spark.sql("""
CREATE TABLE orders_in_parquet(
  order_id INT,
  order_date STRING,
  order_customer_id INT,
  order_status STRING
) 
stored as parquet
""")

In [ ]:
spark.sql("LOAD DATA LOCAL INPATH '/home/forgcpmak/retail_db/orders' INTO TABLE orders_in_parquet")

/*
does not work 
*/

In [ ]:
spark.sql("show databases").show()

In [ ]:
spark.sql("use forgcpmak_sms")

In [ ]:
spark.sql("show tables").show()

In [ ]:
spark.sql("""
CREATE TABLE students (
    student_id INT,
    student_first_name STRING,
    student_last_name STRING,
    student_phone_numbers ARRAY<STRING>,
    student_address STRUCT<street:STRING, city:STRING, state:STRING, zip:STRING>
) STORED AS TEXTFILE
ROW FORMAT
    DELIMITED FIELDS TERMINATED BY '\t'
    COLLECTION ITEMS TERMINATED BY ','
    """)

In [ ]:
spark.sql("DESCRIBE fromatted students").show(truncate = False)

In [ ]:
spark.sql("""
INSERT INTO students VALUES (2, 'Donald', 'Duck', ARRAY('1234567890', '2345678901'), NULL)
""")

In [ ]:
spark.sql("select * from students").show(truncate = False)

In [ ]:
spark.sql("""
INSERT INTO students VALUES 
    (3, 'Mickey', 'Mouse', ARRAY('1234567890', '2345678901'), STRUCT('A Street', 'One City', 'Some State', '12345')),
    (4, 'Bubble', 'Guppy', ARRAY('5678901234', '6789012345'), STRUCT('Bubbly Street', 'Guppy', 'La la land', '45678'))
    
   """)

In [ ]:
spark.sql("describe extended students").show(truncate = False)

In [ ]:
spark.sql("drop database forgcpmak_sms cascade")

In [ ]:
spark.sql("describe extended orders").show(truncate = False)

In [ ]:
spark.sql("describe formatted orders").show(truncate = False)